In [1]:
import json
import os
import pickle
from typing import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa
from keras import backend as K
from keras.callbacks import Callback
from keras.layers import (
    LSTM,
    BatchNormalization,
    Concatenate,
    Conv1D,
    ConvLSTM1D,
    Dense,
    Dropout,
    Flatten,
    Input,
    Layer,
    LayerNormalization,
    MaxPooling1D,
    MultiHeadAttention,
)
from keras.optimizers import Adam, RMSprop
from keras.utils import Sequence
# from keras_tuner.engine.hyperparameters import HyperParameters
# from keras_tuner.tuners import BayesianOptimization, Hyperband
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import (
    BackupAndRestore,
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TensorBoard,
)
from tensorflow.keras.models import Model

from metrics_tf import MBE, SMAPE, pearsons_r, r2

%matplotlib inline

plt.style.use("seaborn-v0_8-darkgrid")

DRIVE_PATH = "/content/drive/MyDrive/TFM"
# PATH = os.path.join(DRIVE_PATH, "data")
PATH = "./data"
SYMBOLS = ["ADA", "BNB", "BTC", "EOS", "ETH", "LTC", "TRX", "VET", "XRP"]


def to_csvf(x):
    return x + "USDT.csv"

2023-10-14 17:27:40.492060: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-14 17:27:40.519430: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-14 17:27:40.982224: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/apollo/anaconda3/envs/tfm/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a

In [2]:
def read_file(symbol: str, tf: str, timestamp_unit: str = "ms") -> pd.DataFrame:
    """
    Reads a CSV file, assigns column names, converts the 'date' column to datetime,
    and sets it as the DataFrame's index.

    Parameters
    ----------
    symbol : str
        The symbol.
    tf : str
        The tf.
    timestamp_unit : str, default 'ms'
        The unit of the timestamp in the 'date' column. By default, it's 'ms' (milliseconds).

    Returns
    -------
    pd.DataFrame
        The DataFrame with the 'date' column converted to datetime and set as the index.
    """

    df = pd.read_csv(to_csvf(os.path.join(PATH, tf, symbol)), header=None).iloc[:, 0:6]
    df.columns = ["date", "open", "high", "low", "close", "volume"]
    df["date"] = pd.to_datetime(df["date"], unit=timestamp_unit)
    df.set_index("date", inplace=True)

    return df


def create_input_tensor(data, lookback=250 * 3):
    inputs = []
    for i in range(0, len(data) - lookback, 3):
        inputs.append(data.iloc[i : i + lookback].values)
        # if i < 24: print(data.iloc[i+lookback])

    return np.array(inputs)


def create_target_tensor(data_dict):
    # Step 1: Create a dataframe with closing prices for each currency
    close_prices_df = pd.DataFrame(
        {symbol: df["close"] for symbol, df in data_dict.items()}
    ).dropna()
    # Step 2: Repeat each value three times
    # print(close_prices_df)
    target_array = np.array(
        close_prices_df.apply(
            lambda x: np.array([item for item in x for _ in range(3)]), axis=1
        ).values
    )

    return np.vstack(target_array)


def prepare_data(PATH):
    # 1. Read BTC hourly data
    btc_data = read_file("BTC", "8h")
    # Create a new index to fill missing values
    full_index = pd.date_range(btc_data.index.min(), btc_data.index.max(), freq="8H")
    df_full = pd.DataFrame(index=full_index)
    # Create the new dataframe forwarding missing values
    btc_data = df_full.merge(
        btc_data, left_index=True, right_index=True, how="left"
    ).fillna(method="ffill")
    # 2. Read other currencies' daily data
    daily_data = {}
    for symbol in SYMBOLS:
        if symbol != "BTC":
            daily_data[symbol] = read_file(f"{symbol}", "1d")

    # 3. Find overlapping date range
    min_date = btc_data.index.min()
    max_date = btc_data.index.max()
    for df in daily_data.values():
        min_date = max(min_date, df.index.min())
        max_date = min(max_date, df.index.max())

    # 4. Prune each dataset to the overlapping range
    btc_data = btc_data.loc[min_date - pd.Timedelta(days=250) : max_date]
    for symbol in daily_data:
        daily_data[symbol] = daily_data[symbol].loc[min_date:max_date]
    # 5. Scale the BTC data and each feature separately
    scalers_btc = {}
    for col in btc_data.columns:
        scaler = MinMaxScaler()
        btc_data[col] = scaler.fit_transform(btc_data[col].values.reshape(-1, 1))
        scalers_btc[col] = scaler

    # Create input tensor from scaled BTC hourly data

    input_tensor = create_input_tensor(btc_data)
    # 6. Scale target data (Close Price) for each currency
    scalers_targets = {}
    scaled_targets = {}
    for symbol, df in daily_data.items():
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(df["close"].values.reshape(-1, 1))
        scaled_targets[symbol] = pd.DataFrame(
            scaled_data, columns=["close"], index=df.index
        )  # Save the scaled data as dataframe
        scalers_targets[symbol] = scaler
    # Create target tensor using scaled data
    train_means = {
        name: np.mean(value.close[int(value.index.size * 0.7) :].values)
        for name, value in scaled_targets.items()
    }
    target_tensors = create_target_tensor(scaled_targets)

    return input_tensor, target_tensors, scalers_btc, scalers_targets, train_means


# Use the function
input_data, target_data, btc_scalers, target_scalers, means = prepare_data(PATH)
#
# target_data = target_data.reshape(-1, 24, 1)
input_data.shape, target_data.shape

((1839, 750, 5), (1839, 24))

In [3]:
class Time2Vec(Layer):
    def __init__(self, output_dim=None, **kwargs):
        self.output_dim = output_dim
        super(Time2Vec, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(
            name="W",
            shape=(input_shape[-1], self.output_dim),
            initializer="uniform",
            trainable=True,
        )
        self.P = self.add_weight(
            name="P",
            shape=(input_shape[1], self.output_dim),
            initializer="uniform",
            trainable=True,
        )
        self.w = self.add_weight(
            name="w", shape=(input_shape[1], 1), initializer="uniform", trainable=True
        )
        self.p = self.add_weight(
            name="p", shape=(input_shape[1], 1), initializer="uniform", trainable=True
        )
        super(Time2Vec, self).build(input_shape)

    def call(self, x):
        original = self.w * x + self.p
        sin_trans = K.sin(K.dot(x, self.W) + self.P)

        return K.concatenate([sin_trans, original], -1)

In [4]:
# Finding the indices for the splits
index_70_percent = int(0.7 * len(input_data))
index_85_percent = int(0.85 * len(input_data))

# Splitting the data into train, validation, and test sets
train_input = input_data[:index_70_percent]
train_target = target_data[:index_70_percent]

valid_input = input_data[index_70_percent:index_85_percent]
valid_target = target_data[index_70_percent:index_85_percent]

test_input = input_data[index_85_percent:]
test_target = target_data[index_85_percent:]

train_input.shape, valid_input.shape, test_input.shape, train_target.shape, valid_target.shape, test_target.shape

((1287, 750, 5),
 (276, 750, 5),
 (276, 750, 5),
 (1287, 24),
 (276, 24),
 (276, 24))

In [5]:
# Metrics common to all outputs
common_metrics = ["mse", pearsons_r, SMAPE, MBE]

# Metrics specific to certain outputs
specific_metrics = {
    f"output_{i}_1_prediction": [r2(mean)] for i, mean in zip(range(8), means.values())
}
# Combine the common and specific metrics
metrics = {
    name: common_metrics + specific_metrics.get(name, []) for name in specific_metrics
}
metrics

{'output_0_1_prediction': ['mse',
  <function metrics_tf.pearsons_r(y_true, y_pred)>,
  <function metrics_tf.SMAPE(y_true, y_pred)>,
  <function metrics_tf.MBE(y_true, y_pred)>,
  <function metrics_tf.r2.<locals>.R2(y_true, y_pred)>],
 'output_1_1_prediction': ['mse',
  <function metrics_tf.pearsons_r(y_true, y_pred)>,
  <function metrics_tf.SMAPE(y_true, y_pred)>,
  <function metrics_tf.MBE(y_true, y_pred)>,
  <function metrics_tf.r2.<locals>.R2(y_true, y_pred)>],
 'output_2_1_prediction': ['mse',
  <function metrics_tf.pearsons_r(y_true, y_pred)>,
  <function metrics_tf.SMAPE(y_true, y_pred)>,
  <function metrics_tf.MBE(y_true, y_pred)>,
  <function metrics_tf.r2.<locals>.R2(y_true, y_pred)>],
 'output_3_1_prediction': ['mse',
  <function metrics_tf.pearsons_r(y_true, y_pred)>,
  <function metrics_tf.SMAPE(y_true, y_pred)>,
  <function metrics_tf.MBE(y_true, y_pred)>,
  <function metrics_tf.r2.<locals>.R2(y_true, y_pred)>],
 'output_4_1_prediction': ['mse',
  <function metrics_tf.pea

In [6]:
def build_model(
    lookback: int = 250 * 3,
    num_transformer_layers: int = 4,
    num_attention_heads: int = 3,
    kernel_size: int = 32,
    conv_layers: int = 3,
    lr: float = 0.000185
):
    
    input_data, target_data, btc_scalers, target_scalers, means = prepare_data(PATH)
    
        # Metrics common to all outputs
    common_metrics = ["mse", pearsons_r, SMAPE, MBE]

    # Metrics specific to certain outputs
    specific_metrics = {
        f"output_{i}_1_prediction": [r2(mean)] for i, mean in zip(range(8), means.values())
    }
    # Combine the common and specific metrics
    metrics = {
        name: common_metrics + specific_metrics.get(name, []) for name in specific_metrics
    }
    
    input_shape = (lookback, 5)  # Assuming lookback is defined

    input_layer = Input(shape=input_shape)
    x = Time2Vec(lookback)(input_layer)

    for _ in range(num_transformer_layers, 0, -1):
        x = MultiHeadAttention(num_heads=num_attention_heads, key_dim=lookback)(x, x)
        x = LayerNormalization()(x)

    encoder_output = x

    # Define output branches
    outputs = []
    losses = {}

    for i in range(8):
        for j in range(conv_layers, 0, -1):
            x = Conv1D(32 * (2**j), kernel_size)(encoder_output)
        x = MaxPooling1D(2)(x)
        x = Flatten()(x)
        x = Dense(32)(x)
        output_1 = Dense(1, name=f"output_{i}_1_prediction")(x)
        output_2 = Dense(1, name=f"output_{i}_2_quantile_05")(x)
        output_3 = Dense(1, name=f"output_{i}_3_quantile_95")(x)
        losses[f"output_{i}_1_prediction"] = "mae"
        losses[f"output_{i}_2_quantile_05"] = tfa.losses.PinballLoss(tau=0.05)
        losses[f"output_{i}_3_quantile_95"] = tfa.losses.PinballLoss(tau=0.95)
        outputs.extend([output_1, output_2, output_3])

    model = Model(inputs=input_layer, outputs=outputs)

    # Compile the model
    model.compile(loss=losses, optimizer=Adam(learning_rate=lr), metrics=metrics)

    return model

In [7]:
def extract_plot_data(model, input_data, targets, scalers):
    # Generate model predictions
    predictions = model.predict(input_data)

    # Convert predictions to numpy array if they aren't already
    if not isinstance(predictions, np.ndarray):
        predictions = np.array(predictions)

    data_dict = {}

    for currency_idx, name in zip(
        range(8), ["ADA", "BNB", "EOS", "ETH", "LTC", "TRX", "VET", "XRP"]
    ):
        # Extract and inverse scale true values for the given currency
        true_values = (
            scalers[name]
            .inverse_transform(targets[:, currency_idx * 3].reshape(-1, 1))
            .flatten()
        )

        # Extract and inverse scale predictions and quantiles
        preds = (
            scalers[name]
            .inverse_transform(predictions[currency_idx * 3].reshape(-1, 1))
            .flatten()
        )
        q5 = (
            scalers[name]
            .inverse_transform(predictions[currency_idx * 3 + 1].reshape(-1, 1))
            .flatten()
        )
        q95 = (
            scalers[name]
            .inverse_transform(predictions[currency_idx * 3 + 2].reshape(-1, 1))
            .flatten()
        )

        data_dict[name] = {
            "true_values": true_values.tolist(),
            "predictions": np.clip(preds, 0, None).tolist(),
            "q5": np.clip(q5, 0, None).tolist(),
            "q95": q95.tolist(),
        }

    return data_dict

In [8]:
# os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/home/apollo/anaconda3/envs/tfm/lib/'
# for i in range(6, 50):
#     model = build_model()
#     history = model.fit(
#         train_input,
#         train_target,
#         validation_data=(valid_input, valid_target),
#         epochs=2000,
#         callbacks=[
#             EarlyStopping(
#                 monitor="val_loss", patience=200, restore_best_weights=True, verbose=1
#             ),
#             ReduceLROnPlateau(monitor="val_loss", patience=50),
#             ModelCheckpoint(
#                 "./tmp/checkpoint/model_checkpoint_{epoch}.hdf5", save_freq=5
#             ),
#             TensorBoard(log_dir="./logs"),
#             BackupAndRestore("tmp/backup"),
#         ],
#     )

#     model.save(f"./train/models/model_trial_{i}.hdf5")

#     history_dict = history.history
#     with open(f"./train/history/history_trial_{i}.pkl", "wb") as pickle_file:  # Use "wb" for write binary mode
#         pickle.dump(history_dict, pickle_file)

#     # Assuming your model is stored in the variable 'model'
#     plot_data = extract_plot_data(model, input_data, target_data, target_scalers)

#     # Store in a JSON file
#     with open(f"./train/predictions/plot_data_trial_{i}.json", "w") as json_file:
#         json.dump(plot_data, json_file)

#     !rm -r tmp

In [9]:
# import matplotlib.pyplot as plt

# # Get the list of metrics from the model
# metrics = [metric for metric in model.metrics_names if not metric.endswith("loss")]

# # Calculate the total number of subplots needed (one for each metric, and one for the loss)
# num_plots = len(metrics) + 1

# plt.figure(figsize=(15, 5 * num_plots))

# # Plotting the loss
# plt.subplot(num_plots, 1, 1)
# plt.plot(history.history["loss"], label="Train Loss")
# plt.plot(history.history["val_loss"], label="Validation Loss")
# plt.title("Loss")
# plt.legend()

# # Plotting the metrics
# for i, metric in enumerate(metrics):
#     plt.subplot(num_plots, 1, i + 2)
#     plt.plot(history.history[metric], label=f"Train {metric}")
#     plt.plot(history.history[f"val_{metric}"], label=f"Validation {metric}")
#     plt.title(metric.capitalize())
#     plt.legend()

# plt.tight_layout()
# plt.show()

# test_results = model.evaluate(test_input, test_target, verbose=1)

In [10]:
# model.output

In [11]:
# Predict for the entire dataset
# all_predictions = model.predict(input_data)

In [12]:
import matplotlib.pyplot as plt


def plot_all_predictions(predictions, targets, test_start_idx, title_prefix):
    # Total length of the data
    total_length = targets.shape[0]

    # Plot each cryptocurrency
    for currency_idx, name in zip(
        range(8), ["ADA", "BNB", "EOS", "ETH", "LTC", "TRX", "VET", "XRP"]
    ):
        # Extract true values for the given currency
        true_values = targets[:, currency_idx * 3]
        l = 0
        # Extract predictions and quantiles
        preds = predictions[currency_idx * 3].reshape(-1)
        print(f"Son iguales {preds == l}")
        q5 = predictions[currency_idx * 3 + 1].reshape(-1)
        q95 = predictions[currency_idx * 3 + 2].reshape(-1)
        l = preds
        plt.figure(figsize=(14, 6))

        # Plot true values and predictions
        plt.plot(true_values, label="True Values", color="black")
        plt.plot(preds, label="Predictions", color="blue")

        # Fill between for the confidence interval
        plt.fill_between(
            range(total_length),
            q5,
            q95,
            color="blue",
            alpha=0.2,
            label="5-95% Confidence Interval",
        )

        # Mark where the test set starts
        plt.axvline(
            x=test_start_idx, color="red", linestyle="--", label="Start of Test Data"
        )

        plt.title(
            f"{name} - Cryptocurrency {currency_idx + 1} Predictions vs True Values"
        )
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()


# Plot for all data
# plot_all_predictions(all_predictions, target_data, len(input_data) - 184, "All Data")

In [13]:
test_target.shape

(276, 24)

In [14]:
import matplotlib.pyplot as plt


def plot_all_predictions(model_preds, targets):
    """
    Plots all prediction values for each currency on the same graph.

    Args:
    - model_preds (list of arrays): List of model outputs. Each entry in the list corresponds to a prediction for a currency.
    - targets (array): True target values for all the currencies.

    """
    num_currencies = (
        len(model_preds) // 3
    )  # Assuming 3 outputs per currency: prediction, 5% quantile, 95% quantile
    plt.figure(figsize=(14, 8))

    # Define colors (if you have more than 8 currencies, extend this list)
    colors = ["b", "g", "r", "c", "m", "y", "k", "orange"]

    for i in range(num_currencies):
        pred = model_preds[i * 3]
        actual = targets[:, i * 3]  # Assuming targets are structured similarly

        # Plotting
        plt.plot(pred, color=colors[i], label=f"Currency {i+1} Prediction")
        # plt.plot(actual, color=colors[i], linestyle='--', label=f'Currency {i+1} Actual')

    plt.legend()
    plt.title("Predictions vs. Actual Values")
    plt.show()


# Usage
# plot_all_predictions(all_predictions, target_data)

In [15]:
# target_data[0]

In [16]:
import matplotlib.pyplot as plt
import numpy as np


def plot_all_predictions(predictions, targets, test_start_idx, title_prefix, scalers):
    # Total length of the data
    total_length = targets.shape[0]

    # Plot each cryptocurrency
    for currency_idx, name in zip(
        range(8), ["ADA", "BNB", "EOS", "ETH", "LTC", "TRX", "VET", "XRP"]
    ):
        # Extract true values for the given currency and inverse scale them
        true_values = (
            scalers[name]
            .inverse_transform(targets[:, currency_idx * 3].reshape(-1, 1))
            .flatten()
        )

        # Extract predictions and quantiles, and inverse scale them
        preds = (
            scalers[name]
            .inverse_transform(predictions[currency_idx * 3].reshape(-1, 1))
            .flatten()
        )
        q5 = (
            scalers[name]
            .inverse_transform(predictions[currency_idx * 3 + 1].reshape(-1, 1))
            .flatten()
        )
        q95 = (
            scalers[name]
            .inverse_transform(predictions[currency_idx * 3 + 2].reshape(-1, 1))
            .flatten()
        )

        plt.figure(figsize=(14, 6))

        # Plot true values and predictions
        plt.plot(true_values, label="True Values", color="black")
        plt.plot(preds, label="Predictions", color="blue")

        # Fill between for the confidence interval
        plt.fill_between(
            range(total_length),
            q5,
            q95,
            color="blue",
            alpha=0.2,
            label="5-95% Confidence Interval",
        )

        # Mark where the test set starts
        plt.axvline(
            x=test_start_idx, color="red", linestyle="--", label="Start of Test Data"
        )
        plt.axvline(
            x=test_start_idx - 184,
            color="yellow",
            linestyle="--",
            label="Start of Validation Data",
        )

        plt.title(
            f"{name} - Cryptocurrency {currency_idx + 1} Predictions vs True Values"
        )
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()


# plot_all_predictions(
#     all_predictions, target_data, len(input_data) - 184, "All Data", target_scalers
# )

In [17]:
def calculate_interval_stats(predictions, targets, test_start_idx):
    # Starting index for the test set
    start_idx = test_start_idx - 184

    stats = {}

    # For each cryptocurrency
    for currency_idx, name in zip(
        range(8), ["ADA", "BNB", "EOS", "ETH", "LTC", "TRX", "VET", "XRP"]
    ):
        # Extract true values for the test set
        true_values = targets[start_idx:, currency_idx * 3]

        # Extract quantiles for the test set
        q5 = predictions[currency_idx * 3 + 1][start_idx:].reshape(-1)
        q95 = predictions[currency_idx * 3 + 2][start_idx:].reshape(-1)

        # Count how many of the true values fall within the confidence interval
        count_inside_interval = np.sum((true_values >= q5) & (true_values <= q95))

        # Count outside the interval
        count_outside_interval = len(true_values) - count_inside_interval

        # Calculate precision
        precision = count_inside_interval / len(true_values)

        stats[name] = {
            "precision": precision,
            "inside_interval": count_inside_interval,
            "outside_interval": count_outside_interval,
        }

    return stats


# # Calculate precision for each cryptocurrency
# stats = calculate_interval_stats(all_predictions, target_data, len(input_data) - 184)

# for name, data in stats.items():
#     print(f"{name}:")
#     print(f"  Precision: {data['precision']:.4f}")
#     print(f"  Inside Interval: {data['inside_interval']}")
#     print(f"  Outside Interval: {data['outside_interval']}\n")

In [ ]:
import os
import numpy as np
import pickle
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

# Set the local directory path
local_dir = "./"

n_folds = 10

def get_train_test_data(input_data, target_data, fold, n_folds):
    fold_size = len(input_data) // n_folds
    test_start = fold * fold_size
    test_end = (fold + 1) * fold_size

    train_input = np.concatenate((input_data[:test_start], input_data[test_end:]))
    test_input = input_data[test_start:test_end]

    train_target = np.concatenate((target_data[:test_start], target_data[test_end:]))
    test_target = target_data[test_start:test_end]

    return train_input, test_input, train_target, test_target

# Loop through each fold

for trial in range(3):  # Train 3 models for each fold
    for fold in range(n_folds):
        input_data, target_data, btc_scalers, target_scalers, means = prepare_data(PATH)

        # Prepare data
        train_input, test_input, train_target, test_target = get_train_test_data(input_data, target_data, fold, n_folds)
        print(f"Train Input shape: {train_input.shape} Test Input shape: {test_input.shape}")
        print(f"Train Target shape: {train_target.shape} Test Target shape: {test_target.shape}")
        # Build and train model
        os.makedirs("./tmp", exist_ok=True)
        os.makedirs("./tmp/backup", exist_ok=True)
        os.makedirs("./tmp/checkpoint", exist_ok=True)
        model = build_model()
        history = model.fit(
            train_input,
            train_target,
            validation_data=(test_input, test_target),
            epochs=2000,
            callbacks=[
                EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True, verbose=1),
                ReduceLROnPlateau(monitor="val_loss", patience=25),
                ModelCheckpoint(os.path.join(local_dir, f"tmp/checkpoint/model_checkpoint_fold_{fold}_trial_{trial}_epoch_{{epoch}}.hdf5"), save_freq=5),
                TensorBoard(log_dir=os.path.join(local_dir, f"logs/fold_{fold}_trial_{trial}")),
                BackupAndRestore(os.path.join(local_dir, f"tmp/backup/fold_{fold}_trial_{trial}"))
            ],
        )
        
        # Save model and history
        os.makedirs(os.path.join(local_dir, f"train/models/fold_{fold}"), exist_ok=True)
        model.save(os.path.join(local_dir, f"train/models/fold_{fold}/model_trial_{trial}.hdf5"))

        history_dict = history.history
        os.makedirs(os.path.join(local_dir, f"train/history/fold_{fold}"), exist_ok=True)
        with open(os.path.join(local_dir, f"train/history/fold_{fold}/history_trial_{trial}.pkl"), "wb") as pickle_file:
            pickle.dump(history_dict, pickle_file)

        # Extract plot data
        plot_data = extract_plot_data(model, input_data, target_data, target_scalers)  # Assuming target_scalers is defined

        # Save plot data
        os.makedirs(os.path.join(local_dir, f"train/predictions/fold_{fold}"), exist_ok=True)
        with open(os.path.join(local_dir, f"train/predictions/fold_{fold}/plot_data_trial_{trial}.json"), "w") as json_file:
            json.dump(plot_data, json_file)

        # Remove temporary files
        os.system(f"rm -r ./tmp")


Train Input shape: (1656, 750, 5) Test Input shape: (183, 750, 5)
Train Target shape: (1656, 24) Test Target shape: (183, 24)
Epoch 1/2000
 4/52 [=>............................] - ETA: 16s - loss: 203.0067 - output_0_1_prediction_loss: 18.4004 - output_0_2_quantile_05_loss: 17.5346 - output_0_3_quantile_95_loss: 20.8978 - output_1_1_prediction_loss: 11.7716 - output_1_2_quantile_05_loss: 2.1443 - output_1_3_quantile_95_loss: 3.0852 - output_2_1_prediction_loss: 15.6322 - output_2_2_quantile_05_loss: 6.8391 - output_2_3_quantile_95_loss: 0.5546 - output_3_1_prediction_loss: 9.5976 - output_3_2_quantile_05_loss: 4.1904 - output_3_3_quantile_95_loss: 2.3342 - output_4_1_prediction_loss: 15.5959 - output_4_2_quantile_05_loss: 0.5219 - output_4_3_quantile_95_loss: 3.0676 - output_5_1_prediction_loss: 11.4441 - output_5_2_quantile_05_loss: 2.0562 - output_5_3_quantile_95_loss: 2.1328 - output_6_1_prediction_loss: 19.8653 - output_6_2_quantile_05_loss: 5.2953 - output_6_3_quantile_95_loss: 3.

/home/apollo/anaconda3/envs/tfm/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


52/52 [==============================] - 33s 488ms/step - loss: 171.8052 - output_0_1_prediction_loss: 14.3312 - output_0_2_quantile_05_loss: 4.4655 - output_0_3_quantile_95_loss: 5.8169 - output_1_1_prediction_loss: 11.6961 - output_1_2_quantile_05_loss: 2.3454 - output_1_3_quantile_95_loss: 2.7505 - output_2_1_prediction_loss: 12.5038 - output_2_2_quantile_05_loss: 3.0085 - output_2_3_quantile_95_loss: 4.2327 - output_3_1_prediction_loss: 19.0053 - output_3_2_quantile_05_loss: 4.1557 - output_3_3_quantile_95_loss: 2.2662 - output_4_1_prediction_loss: 13.8132 - output_4_2_quantile_05_loss: 0.6868 - output_4_3_quantile_95_loss: 1.5231 - output_5_1_prediction_loss: 17.8687 - output_5_2_quantile_05_loss: 4.0252 - output_5_3_quantile_95_loss: 4.8746 - output_6_1_prediction_loss: 19.0718 - output_6_2_quantile_05_loss: 2.3349 - output_6_3_quantile_95_loss: 3.3289 - output_7_1_prediction_loss: 14.7103 - output_7_2_quantile_05_loss: 1.5738 - output_7_3_quantile_95_loss: 1.4161 - output_0_1_pr